# Recent advancements in Histogramming
## Henry Schreiner, Aman Geol, and Nino Lau


Run the code with us through Binder, altering examples and asking "what if" questions along the way :)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/henryiii/histogram-tutorial/master?filepath=talk_1_bh_uhi.ipynb)

# A new era: boost-histogram 1.0!

Boost-histogram 1.0 came out with some great features:

* Removal of Python 2.7 & 3.5 support
    * Fewer lines of code and more features than 0.13!
* Fully staticly typed
* Easier subclassing

Along with many great features available in 0.11-0.13:
* Much simpler, Pythonic metadata system
* Much better access to accumulator storages
* PlottableProtocol support! (see UHI section later)

Huge updates to pybind11 & cibuildwheel as part of this work, too!
* New CMake and setuptools helpers
* 40% faster accumulation with less memory used
* Python 3.9 support
* And much more!

# Demo: statically typed histogram

# UHI: Unified Histotogram Interface

A staticly typed protocol for plotting histograms from any source.

Also an actual PyPI library, with helper functions, and lots of centralized documentation.

# Demo: Plotting a boost-histogram with mplhep or histoprint